In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sys
sys.path.append('../validation/')
import  photo_z_validation as phz 
import pandas as pd
import seaborn as sns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Fist lets make some fake data, we will make 1000 PDF's with each a 100 values and some dummy z_spec and dummy weights and put it in a Pandas DataFrame.

In [2]:
a = np.random.random_integers(1, 100, 100)
s = np.random.dirichlet(a, 1000)
z = np.linspace(0, 2.0, 100)
pdf_names = ['pdf_' + str(i) for i in range(100)]
df_test = pd.DataFrame(s, columns=pdf_names)
df_test['z_spec'] = np.random.uniform(0.0, 2.0, 1000)
df_test['weight'] = np.random.dirichlet(a, 1000)[:,0]

In [3]:
df_test.head()

,pdf_0,pdf_1,pdf_2,pdf_3,pdf_4,pdf_5,pdf_6,pdf_7,pdf_8,pdf_9,...,pdf_92,pdf_93,pdf_94,pdf_95,pdf_96,pdf_97,pdf_98,pdf_99,z_spec,weight
0,0.008616,0.006788,0.004494,0.010290,0.001118,0.004449,0.013578,0.011239,0.009508,0.005664,...,0.004060,0.017868,0.007753,0.018682,0.014335,0.007689,0.017735,0.009086,1.032741,0.008451
1,0.007770,0.007435,0.006430,0.010512,0.000298,0.006679,0.015038,0.012488,0.008757,0.004787,...,0.003899,0.017178,0.005388,0.013499,0.015864,0.008860,0.011491,0.010369,1.677347,0.008098
2,0.009266,0.004525,0.003903,0.010710,0.001134,0.005486,0.011947,0.012130,0.012990,0.004699,...,0.005141,0.018830,0.006155,0.014556,0.018736,0.008478,0.014200,0.011975,0.435301,0.008343
3,0.006333,0.005320,0.004406,0.011551,0.001593,0.004118,0.012137,0.009927,0.014889,0.003056,...,0.003972,0.017543,0.006097,0.016056,0.018455,0.008772,0.013167,0.011053,1.944341,0.007741
4,0.010547,0.006757,0.003019,0.007961,0.001442,0.006127,0.013560,0.014540,0.011404,0.005567,...,0.004779,0.015685,0.004538,0.016677,0.012561,0.009781,0.016960,0.009791,1.840181,0.008974


The validation code expect a pandas data frame in wich the pdf values are named pdf_i with i a number between 0 and 500. In this example we have 100 values. The DataFrame will also need a 'z_spec' value. 

The Following function will calculate the mean of n-tomographics bins that the user can define. It also returns the weighted means of the spectroscopic sample and of the photometric redshift. You also need to to pass a photo-z on wich the binning will be made, this allows to look at performance when binned on z from different code. All values come with bootsrtap error-bars, which works quite fast. 

```Python
    #:param df: pandas data-frame
    #:param binning: center of redshift bins
    #:param metric : 'mean','mode' of the pdf as the point estimate, default = 'mean'
    #:param weights: optional weighting scheme with same len as df
    #:param tomo_bins: in which z-bins array example [0.0, 0.2, 0.6, 1.8]
    #:param n_resample : Amount of resamples to estimate mean and variance on the mean default 50.
    #:param z_phot : Numpy array of photo-z values on wich to make binning
    #:return: pandas data frame with mean estimates
```

In [4]:
tomo_bins = [0.0, 0.3, 0.6, 0.9, 1.3, 2.0]
a = phz.mean(df = df_test, binning= z, weights='weight', tomo_bins= tomo_bins, z_phot = np.random.uniform(0,2.0,len(df_test)))
a

6


,mean_z_bin,n_obj,mean_spec,err_mean_spec,mean_phot,err_mean_phot,w_mean_spec,w_err_mean_spec,w_mean_phot,w_err_mean_phot
"z [0.0, 0.3]",0.15,177,0.938453,0.049635,1.040923,0.000504,0.922634,0.038394,1.041131,0.000435
"z [0.3, 0.6]",0.45,157,1.016809,0.042666,1.041001,0.000617,1.013595,0.043818,1.040820,0.000498
"z [0.6, 0.9]",0.75,158,1.018675,0.041551,1.041182,0.000565,1.016844,0.042856,1.041118,0.000567
"z [0.9, 1.3]",1.10,176,1.019639,0.043271,1.040364,0.000562,1.002651,0.039968,1.040444,0.000543
"z [1.3, 2.0]",1.65,332,1.016717,0.030998,1.040595,0.000418,1.026530,0.028351,1.040671,0.000405


In [5]:
# example to get in many bins 
tomo_bins = np.linspace(0,2.0, 40)
a = phz.mean(df = df_test, binning= z, weights='weight', tomo_bins= tomo_bins, z_phot = np.random.uniform(0,2.0,len(df_test)))
a

40


,mean_z_bin,n_obj,mean_spec,err_mean_spec,mean_phot,err_mean_phot,w_mean_spec,w_err_mean_spec,w_mean_phot,w_err_mean_phot
"z [0.0, 0.0512820512821]",0.025641,20,0.828692,0.127672,1.040743,0.001284,0.868894,0.124972,1.040514,0.001712
"z [0.0512820512821, 0.102564102564]",0.076923,30,1.039519,0.095308,1.038988,0.001346,1.060746,0.087906,1.039424,0.001436
"z [0.102564102564, 0.153846153846]",0.128205,29,0.942743,0.111081,1.041094,0.001593,0.866701,0.115900,1.040985,0.001323
"z [0.153846153846, 0.205128205128]",0.179487,19,1.012040,0.146070,1.039942,0.001599,1.020688,0.150502,1.040646,0.001399
"z [0.205128205128, 0.25641025641]",0.230769,19,0.944235,0.155024,1.043534,0.001483,0.972694,0.116464,1.043188,0.001163
"z [0.25641025641, 0.307692307692]",0.282051,26,0.974728,0.126722,1.039725,0.001288,0.974600,0.104846,1.039619,0.001255
"z [0.307692307692, 0.358974358974]",0.333333,30,1.093716,0.093986,1.041762,0.001293,1.104503,0.102139,1.041720,0.001099
"z [0.358974358974, 0.410256410256]",0.384615,33,1.070718,0.094949,1.040917,0.001310,1.055789,0.113038,1.040906,0.000906
"z [0.410256410256, 0.461538461538]",0.435897,20,1.218451,0.128844,1.040667,0.001497,1.138256,0.135407,1.040182,0.001387
"z [0.461538461538, 0.512820512821]",0.487179,18,1.080723,0.131663,1.041318,0.001721,1.019911,0.121613,1.041508,0.001448


The following script creates weighted photo-z and spec distributions in user defined tomo-bins.  

In [6]:
tomo_bins = [0.0, 0.3, 0.6, 0.9, 1.3, 2.0]
phot_nz, spec_nc, for_WL = phz.weighted_nz_distributions(df = df_test, binning= z, weights='weight', tomo_bins= tomo_bins, z_phot = np.random.uniform(0,2.0,len(df_test)))

```Python
    phot_nz[0][0] countains full non tomographic weighted n(z) estimate
    phot_nz[0][1] countains full non tomographic weighted n(z) estimate 1st bootsrap resample
    phot_nz[0][2] countains full non tomographic weighted n(z) estimate 2nd bootsrap resample

    phot_nz[1][0] countains full first tomographic bin weighted n(z) estimate
    phot_nz[1][1] countains full first tomographic bin weighted n(z) estimate 1st bootsrap resample
    phot_nz[1][2] countains full first tomographic bin weighted n(z) estimate 2nd bootsrap resample
    .
    .
    .
    spec_nz[0][0] countains full non tomographic weighted spec-z n(z) estimate
    spec_nz[0][1] countains full non tomographic weighted spec-z n(z) estimate 1st bootsrap resample
    spec_nz[0][2] countains full non tomographic weighted spec-z n(z) estimate 2nd bootsrap resample

    spec_nz[1][0] countains full first tomographic bin weighted spec-z n(z) estimate
    spec_nz[1][1] countains full first tomographic bin weighted spec-z n(z) estimate 1st bootsrap resample
    spec_nz[1][2] countains full first tomographic bin weighted spec-x n(z) estimate 2nd bootsrap resample
    
    for_WL is a Dictionary that is need to pass to the WL paramter estimation code by Troxel. 
```

In [7]:
# you can save the for_WL to pickle file like this
phz.ld_writedicts('Filename.pickle',for_WL)